# Import Library

In [1]:
!pip install langchain google-cloud-aiplatform qdrant-client cohere wikipedia -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.0 MB/s et

# Google Authentication

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from getpass import getpass
project_id = getpass()

··········


In [4]:
import vertexai
import google.auth
from vertexai.preview.language_models import TextGenerationModel

creds, _ = google.auth.default(quota_project_id=project_id)
vertexai.init(project=project_id, credentials=creds)

# Load LLM

In [28]:
from langchain.llms import VertexAI
llm = VertexAI(model_name="text-bison", max_output_tokens=1024, temperature=0.3)

## Basic Example

In [6]:
print(llm("한국의 수도는?"))

 서울


In [7]:
print(llm("이대호는 누구야?"))

 이대호(1982년 6월 11일 ~ )는 대한민국의 야구 선수로, KBO 리그 삼성 라이온즈의 포수이다.

이대호는 2001년 신인 드래프트에서 1차 지명을 받아 삼성 라이온즈에 입단했다. 2003년에 데뷔한 이대호는 2006년에 첫 골든 글러브를 수상했고, 2009년에는 한국 프로 야구 역사상 최초로 포수로서 홈런왕을 차지했다. 이대호는 2010년에 메이저 리그 시애틀 매리너스로 이적했지만, 2012년에 삼성 라이온즈로 복귀했다.

이대호는 2013년에 KBO 리그 MVP를 수상했고, 2014년에는 한국 프로 야구 역사상 최초로 포수로서 타점왕을 차지했다. 이대호는 2015년에 일본 프로 야구 소프트뱅크 호크스로 이적했지만, 2017년에 삼성 라이온즈로 복귀했다.

이대호는 2018년에 KBO 리그 홈런왕을 차지했고, 2019년에는 한국 프로 야구 역사상 최초로 포수로서 타율왕을 차지했다. 이대호는 2020년에 삼성 라이온즈와 재계약을 맺었고, 현재까지도 삼성 라이온즈에서 활약하고 있다.

이대호는 한국 프로 야구 역사상 최고의 포수 중 한 명으로 평가받고 있다. 이대호는 강한 타격력과 수비력을 겸비하고 있으며, 리더십도 뛰어나다. 이대호는 한국 프로 야구에서 많은 기록을 수립했고, 한국 프로 야구의 발전에 크게 기여했다.


In [8]:
print(llm("손아섭의 2014년 홈런 개수는?"))

 손아섭은 2014년에 은퇴했기 때문에 2014년에 홈런을 기록하지 않았습니다.


# Load CSV

In [9]:
import pandas as pd

df = pd.read_csv("./hitter_record_1982_to_2023_cleaned(ver.2).csv")

In [10]:
from langchain.schema import Document

documents = []
for name in df['이름'].unique():
    document = f"{name} 선수에 관한 문서입니다. \n"
    for idx, row in df[df['이름'] == name].iterrows():
        document += str(row.to_dict())[1:-1]
        document += '\n'

    documents.append(Document(page_content=document))

# Cohere API Authentication

In [11]:
from getpass import getpass
cohere_api_key = getpass()

··········


# Save DB

In [12]:
from langchain.embeddings import CohereEmbeddings
from langchain.vectorstores import Qdrant

embeddings = CohereEmbeddings(model="embed-multilingual-light-v3.0", cohere_api_key=cohere_api_key)

db = Qdrant.from_documents(
    documents,
    embeddings,
    path="/tmp/local_qdrant",
    collection_name="my_documents",
    )


# Practice

## 1. Simple LCEL Practice

In [29]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 템플릿
template = '''아래 글만을 참고해서 질문에 대한 답을 해줘.
            {context}

            질문: {question}

            답변:
'''
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 1})

# 프롬프트 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=["question", "context"])

# LLM Chain 객체 생성
hitter_detail_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### Example

In [14]:
question = "손아섭의 2014년 홈런 개수는?"
hitter_detail_chain.invoke(question)

' 손아섭의 2014년 홈런 개수는 18개입니다.'

In [15]:
question = "손아섭의 2010년 기록을 알려줘"
hitter_detail_chain.invoke(question)

' 손아섭의 2010년 기록은 다음과 같습니다.\n\n- 소속팀: 롯데 자이언츠\n- 포지션: LF\n- 타율: 0.306\n- 홈런: 11개\n- 타점: 47개\n- 도루: 6개\n- WAR: 2.85'

In [54]:
question = "호세가 출루율 0.5를 넘었던 시즌은?"
hitter_detail_chain.invoke(question)

' 호세가 출루율 0.5를 넘었던 시즌은 2001년입니다.'

In [55]:
question = "아두치가 20-20을 달성한 시즌은?"
hitter_detail_chain.invoke(question)

' 아두치가 20-20을 달성한 시즌은 2015년입니다.'

In [56]:
question = "테임즈가 40-40을 달성한 시즌은? 기록과 함께 알려줘. 그리고 40-40의 의미는 뭐야?"
hitter_detail_chain.invoke(question)

' 테임즈가 40-40을 달성한 시즌은 2015 시즌이다. 이 시즌에 테임즈는 47개의 홈런과 40개의 도루를 기록했다.\n\n40-40은 한 시즌에 40개 이상의 홈런과 40개 이상의 도루를 기록하는 것을 말한다. 이는 매우 어려운 기록으로, 메이저 리그 역사상 40-40을 달성한 선수는 4명 뿐이다. 테임즈는 KBO 역사상 최초로 40-40을 달성한 선수이다.'

In [53]:
question = "이승엽이 홈런을 가장 많이 친 시즌은?"
hitter_detail_chain.invoke(question)

' 이승엽이 홈런을 가장 많이 친 시즌은 2003년으로, 56개의 홈런을 기록했습니다.'

In [16]:
question = "문규현은 원클럽맨이니? 그 이유는?"
hitter_detail_chain.invoke(question)

' 문규현은 원클럽맨입니다. 그는 2002년부터 2019년까지 롯데 자이언츠에서만 뛰었습니다.'

In [17]:
question = "손아섭은 KBO에서 원클럽맨이니? 그 이유는?"
hitter_detail_chain.invoke(question)

' 손아섭은 KBO에서 원클럽맨이 아닙니다. 그는 2022년에 NC 다이노스로 이적했습니다.'

In [18]:
question = "안치홍이 뛰었던 팀들을 얘기해줄래?"
hitter_detail_chain.invoke(question)

' 안치홍이 뛰었던 팀은 다음과 같습니다.\n\n- KIA 타이거즈\n- 롯데 자이언츠'

In [19]:
question = "안치홍의 주 포지션은 어디야?"
hitter_detail_chain.invoke(question)

' 안치홍의 주 포지션은 2루수입니다.'

In [20]:
question = "이대호에 대해 알려줄래?"
hitter_detail_chain.invoke(question)

' 이대호는 1982년 6월 22일생으로, 롯데 자이언츠에서 뛰는 야구 선수입니다. 2001년에 프로에 데뷔하여 2022년까지 롯데 자이언츠에서만 뛰고 있습니다. 주로 1루수와 지명타자로 활약했으며, 2010년에는 홈런왕과 타점왕을 차지하기도 했습니다. 이대호는 통산 2,500안타를 달성한 역대 11번째 선수이며, 2017년에는 월드 베이스볼 클래식에 국가대표로 출전하기도 했습니다.'

In [21]:
question = "이승엽은 누구야?"
hitter_detail_chain.invoke(question)

' 질문: 이승엽은 누구야?\n\n답변:\n이승엽은 전 KBO 리그 삼성 라이온즈의 야구 선수입니다. 그는 1995년부터 2017년까지 프로 야구 선수로 활약했습니다. 그는 1998년과 2003년에 홈런왕을 차지했고, 2003년에는 타점왕을 차지했습니다. 그는 2006년에 두산 베어스로 이적했지만, 2012년에 삼성 라이온즈로 복귀했습니다. 그는 2017년에 은퇴했습니다.'

## 2. Simple Agent Practice

In [31]:
from langchain.retrievers import WikipediaRetriever

retriever = WikipediaRetriever(lang='ko', search_type="mmr")

# LLM Chain 객체 생성
wiki_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | VertexAI(model_name="text-bison", max_output_tokens=1024, temperature=0.9)
    | StrOutputParser()
)

In [32]:
from langchain.agents import Tool

tools = [
    Tool(
        name="타자 시즌별 기록",
        func=hitter_detail_chain.invoke,
        description="해당 타자의 시즌별 기록을 알아볼 때 유용합니다.",
    ),

    Tool.from_function(
        name="위키피디아",
        func= wiki_chain.invoke,
        description="해당 타자의 전반적인 정보를 알고 싶을 때 유용합니다."
    )
]

In [33]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [34]:
agent.invoke(
    {
        "input": "이대호가 누구야?"
    }
)



> Entering new AgentExecutor chain...
 이대호에 대한 정보를 알고 싶으니 위키피디아에서 찾아보자.
Action: 위키피디아
Action Input: 이대호
Observation:  이대호(李大浩, 1982년 6월 21일~)는 대한민국의 야구인이다. 현역 시절 KBO 리그의 롯데 자이언츠, 일본 프로 야구의 오릭스 버펄로스, 후쿠오카 소프트뱅크 호크스, 메이저 리그 베이스볼의 시애틀 매리너스에서 내야수, 포수, 투수, 지명 타자로 활동했다.
Thought: 이대호는 1982년 6월 21일에 태어난 대한민국의 야구 선수입니다. 그는 KBO 리그의 롯데 자이언츠, 일본 프로 야구의 오릭스 버펄로스, 후쿠오카 소프트뱅크 호크스, 메이저 리그 베이스볼의 시애틀 매리너스에서 내야수, 포수, 투수, 지명 타자로 활동했습니다.
Final Answer: 이대호는 1982년 6월 21일에 태어난 대한민국의 야구 선수입니다. 그는 KBO 리그의 롯데 자이언츠, 일본 프로 야구의 오릭스 버펄로스, 후쿠오카 소프트뱅크 호크스, 메이저 리그 베이스볼의 시애틀 매리너스에서 내야수, 포수, 투수, 지명 타자로 활동했습니다.

> Finished chain.


{'input': '이대호가 누구야?',
 'output': '이대호는 1982년 6월 21일에 태어난 대한민국의 야구 선수입니다. 그는 KBO 리그의 롯데 자이언츠, 일본 프로 야구의 오릭스 버펄로스, 후쿠오카 소프트뱅크 호크스, 메이저 리그 베이스볼의 시애틀 매리너스에서 내야수, 포수, 투수, 지명 타자로 활동했습니다.'}

In [35]:
agent.invoke(
    {
        "input": "이승엽은 2003시즌 홈런을 몇개 기록했지?"
    }
)



> Entering new AgentExecutor chain...
 이승엽의 2003시즌 홈런 기록을 알아보고 싶다.
Action: 타자 시즌별 기록
Action Input: 이승엽 2003
Observation:  - 소속팀: 삼성 라이온즈
- 타율: 0.301
- 홈런: 56
- 타점: 144
- WAR: 8.39
Thought: 이승엽의 2003시즌 홈런 기록은 56개다.
Final Answer: 56

> Finished chain.


{'input': '이승엽은 2003시즌 홈런을 몇개 기록했지?', 'output': '56'}

In [39]:
question = "이대형의 2010시즌 도루개수에 대해 알려줄래?"

agent.invoke(
    {
        "input": question
    }
)



> Entering new AgentExecutor chain...
 이대형의 2010시즌에 대한 정보를 알고 싶은 것 같다.
Action: 타자 시즌별 기록
Action Input: 이대형 2010
Observation:  이대형 선수의 2010년도 성적은 다음과 같습니다.

소속팀: LG 트윈스
포지션: CF
WAR*: 1.91
G: 130
타석: 568
타수: 494
득점: 73
안타: 129
2타: 14
3타: 2
홈런: 1
루타: 150
타점: 43
도루: 66
도실: 21
볼넷: 58
사구: 2
고4: 0
삼진: 76
병살: 3
희타: 13
희비: 1
타율: 0.261
출루: 0.341
장타: 0.304
OPS: 0.644
wOBA: 0.305
wRC+: 75.0
WPA: nan
Thought: 이대형 선수의 2010년도 도루 개수는 66개 입니다.
Final Answer: 이대형 선수의 2010년도 도루 개수는 66개 입니다.

> Finished chain.


{'input': '이대형의 2010시즌 도루개수에 대해 알려줄래?',
 'output': '이대형 선수의 2010년도 도루 개수는 66개 입니다.'}